In [ ]:
import os
import json
from tqdm import tqdm
from typing import List, Dict, Any
from ultralytics import YOLO

def process_images(directory: str, model: YOLO, output_file: str) -> None:
    """
    Reads images from a directory, runs inference using a model,
    and saves results in the specified format to an output file.

    Args:
        directory (str): Path to the directory containing images.
        model (YOLO): Pre-trained YOLO model for inference.
        output_file (str): Path to save the results.
    """
    # Get a sorted list of image files in the directory
    image_extensions = ('.png', '.jpg', '.jpeg')
    files_in_directory = sorted(
        [os.path.join(directory, f) for f in os.listdir(directory) if f.lower().endswith(image_extensions)]
    )

    # Open the output file to save results
    with open(output_file, 'w') as outfile:
        for frame_path in tqdm(files_in_directory, desc="Processing frames"):
            # Run inference on the frame
            results = model([frame_path])  # Replace with your model's inference method

            # Extract the filename and frame number
            filename = os.path.basename(frame_path)
            frame_number = int(filename.split('_')[1].split('.')[0])

            # Collect bounding box data
            frame_results = []
            for result in results:
                boxes = result.boxes
                for box, conf in zip(boxes.xyxy.tolist(), boxes.conf.tolist()):
                    x1, y1, x2, y2 = [round(coord, 3) for coord in box]  # Round each coordinate
                    frame_results.append([x1, y1, x2, y2, conf])

            # Write results in the desired format
            result_data = {frame_number: frame_results}
            outfile.write(json.dumps(result_data) + '\n')

def load_model(model_path: str) -> YOLO:
    """
    Loads a YOLO model from the specified path.

    Args:
        model_path (str): Path to the model weights.

    Returns:
        YOLO: Loaded YOLO model.
    """
    try:
        model = YOLO(model_path, verbose=False)
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model from {model_path}: {e}")

def ensure_directory_exists(directory: str) -> None:
    """
    Ensures that the specified directory exists. If not, it creates the directory.

    Args:
        directory (str): Path to the directory.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

def main() -> None:
    # Define paths and model name
    model_name = "yolo11n_DJI_0010_V_2_Aug@fine-tuning"
    video_name = "DJI_20240308111117_0010_V_1"
    model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{model_name}/best.pt"
    image_directory = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/valid/images"

    # Define output directory and ensure it exists
    output_directory = "./output_results"
    ensure_directory_exists(output_directory)

    # Generate output file name using model name and video name
    output_results_file = os.path.join(output_directory, f"results_{model_name}_{video_name}.txt")

    # Load the model
    model = load_model(model_path)

    # Process images and save results
    process_images(image_directory, model, output_results_file)

In [ ]:
main()